In [221]:
# define custom filter bank and load images
import pywt
from pywt import dwt2, idwt2, dwt, idwt, wavedec
import cv2
import math
import numpy as np

class NewFilterBank(object):
  @property
  def filter_bank(self):
    rt = math.sqrt(4/9)
    dec_lo, dec_hi, rec_lo, rec_hi = [rt, -rt*0.25, rt*0.25, rt], [rt, rt*0.25, rt*0.25, -rt], [rt, rt*0.25, -rt*0.25, rt], [-rt, rt*0.25, rt*0.25, rt]
    return [dec_lo, dec_hi, rec_lo, rec_hi]
filter_bank = NewFilterBank()
newWavelet = pywt.Wavelet(name="newWavelet", filter_bank=filter_bank)

lena_image = 'drive/My Drive/lena_grayscale.jpg'
barbara_image = 'drive/My Drive/barbara_grayscale.png'
img = cv2.imread(lena_image, cv2.COLOR_BGR2GRAY)

In [223]:
# perform DWT and IDWT
# 1D wavelet transform
(L, H) = dwt(img, newWavelet)
decomp_1d = np.hstack((L, H))
idwt_1d = idwt(L, H, newWavelet)
cv2.imwrite('decomp_1d.png', decomp_1d)
cv2.imwrite('idwt_1d.png', idwt_1d)

# 2D wavelet transform
# first level decomposition
LL3, (HL3, LH3, HH3) = dwt2(img, newWavelet) 
cv2.imwrite('LL3.jpg', LL3)
cv2.imwrite('HL3.jpg', HL3)
cv2.imwrite('LH3.jpg', LH3)
cv2.imwrite('HH3.jpg', HH3)
idwt_1st_2d = idwt2((LL3, (HL3, LH3, HH3)), newWavelet)
cv2.imwrite('idwt_1st_2d.jpg', idwt_1st_2d)

# second level decomposition
LL2, (HL2, LH2, HH2) = dwt2(LL3, newWavelet) 
cv2.imwrite('LL2.jpg', LL2)
cv2.imwrite('HL2.jpg', HL2)
cv2.imwrite('LH2.jpg', LH2)
cv2.imwrite('HH2.jpg', HH2)
idwt_2nd_2d = idwt2((LL2, (HL2, LH2, HH2)), newWavelet)
cv2.imwrite('idwt_2nd_2d.jpg', idwt_2nd_2d)

# third level decomposition
LL1, (HL1, LH1, HH1) = dwt2(LL2, newWavelet) 
cv2.imwrite('LL1.jpg', LL1)
cv2.imwrite('HL1.jpg', HL2)
cv2.imwrite('LH1.jpg', LH2)
cv2.imwrite('HH1.jpg', HH2)
idwt_3rd_2d = idwt2((LL1, (HL1, LH1, HH1)), newWavelet)
cv2.imwrite('idwt_3rd_2d.jpg', idwt_3rd_2d)

True

In [215]:
# stacking the images together
# 1D level
vert_1d = np.hstack((L, H))
cv2.imwrite('vert_1d.jpg', vert_1d)

# 1st level
top_1st = np.hstack((LL3, HL3))
bottom_1st = np.hstack((LH3, HH3))
decomp_1st = np.vstack((top_1st, bottom_1st))
cv2.imwrite('decomp_1st.jpg', decomp_1st)

# 2nd level
top_2nd = np.hstack((LL2, HL2))
bottom_2nd = np.hstack((LH2, HH2))
decomp_2nd = np.vstack((top_2nd, bottom_2nd))
cv2.imwrite('decomp_2nd.jpg', decomp_2nd)

# 3rd level
top_3rd = np.hstack((LL1, HL1))
bottom_3rd = np.hstack((LH1, HH1))
decomp_3rd = np.vstack((top_3rd, bottom_3rd))
cv2.imwrite('decomp_3rd.jpg', decomp_3rd)

True

In [216]:
# compute energy of subband

def energy(arr):
  energy = np.sum(np.square(arr))
  return round(energy, 3)

# total energy
total_ = energy(LL3) + energy(HL3) + energy(LH3) + energy(HH3) 
+ energy(LL2) + energy(HL2) + energy(LH2) + energy(HH2) 
+ energy(LL1) + energy(HL1) + energy(LH1) + energy(HH1)

print("Total energy:", total_)
print("LL3:", energy(LL3), "Percentage:", (energy(LL3)/total_)*100)
print("HL3:", energy(HL3), "Percentage:", (energy(HL3)/total_)*100)
print("LH3:", energy(LH3), "Percentage:", (energy(LH3)/total_)*100)
print("HH3:", energy(HH3), "Percentage:", (energy(HH3)/total_)*100)
print("LL2:", energy(LL2), "Percentage:", (energy(LL2)/total_)*100)
print("HL2:", energy(HL2), "Percentage:", (energy(HL2)/total_)*100)
print("LH2:", energy(LH2), "Percentage:", (energy(LH2)/total_)*100)
print("HH2:", energy(HH2), "Percentage:", (energy(HH2)/total_)*100)
print("LL1:", energy(LL1), "Percentage:", (energy(LL1)/total_)*100)
print("HL1:", energy(HL1), "Percentage:", (energy(HL1)/total_)*100)
print("LH1:", energy(LH1), "Percentage:", (energy(LH1)/total_)*100)
print("HH1:", energy(HH1), "Percentage:", (energy(HH1)/total_)*100)

Total energy: 3947385928.818
LL3: 3428123057.576 Percentage: 86.84539893981213
HL3: 248339186.7 Percentage: 6.291231492897436
LH3: 238246431.493 Percentage: 6.035549495013279
HH3: 32677253.049 Percentage: 0.8278200722771698
LL2: 2717241014.547 Percentage: 68.83646705810311
HL2: 184337019.185 Percentage: 4.669850440496393
LH2: 195005265.78 Percentage: 4.9401114888807465
HH2: 19953038.458 Percentage: 0.50547473233697
LL1: 2158223937.582 Percentage: 54.67476391973296
HL1: 150139479.734 Percentage: 3.803516616855286
LH1: 165009247.402 Percentage: 4.180215727004179
HH1: 17190797.107 Percentage: 0.43549826181164886


In [217]:
# calculate MSE of compressed image with original image
import numpy as np

def mse(imageA, imageB):
  imageA, imageB = np.array(imageA), np.array(imageB)
  err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
  err /= float(imageA.shape[0] * imageA.shape[1])

  return err

# convert image format for cv2
def pil2cv(image):
  new_image = np.array(image, dtype=np.uint8)
  if new_image.ndim == 2: 
      pass
  elif new_image.shape[2] == 3:
      new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
  elif new_image.shape[2] == 4:
      new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
  return new_image

# calculate PSNR
def psnr(imageA, imageB):
  psnr = cv2.PSNR(imageA, imageB)
  return psnr

# perform quick DWT with levels for easy reconstruction
def dwt(img, wavelet, level):
  coeffs = pywt.wavedec2(img, wavelet, level)
  inversed = pywt.waverec2(coeffs, wavelet)
  return inversed

idwt_1st_2d = dwt(img, newWavelet, 1)
idwt_2nd_2d = dwt(img, newWavelet, 2)
idwt_3rd_2d = dwt(img, newWavelet, 3)

print("MSE for 1st level decomposition:", mse(idwt_1st_2d, img), "PSNR:", psnr(pil2cv(idwt_1st_2d), img))
print("MSE for 2nd level decomposition:", mse(idwt_2nd_2d, img), "PSNR:", psnr(pil2cv(idwt_2nd_2d), img))
print("MSE for 3rd level decomposition:", mse(idwt_3rd_2d, img), "PSNR:", psnr(pil2cv(idwt_3rd_2d), img))

cv2.imwrite("idwt_1st_2d.jpg", idwt_1st_2d)
cv2.imwrite("idwt_2nd_2d.jpg", idwt_2nd_2d)
cv2.imwrite("idwt_3rd_2d.jpg", idwt_3rd_2d)

MSE for 1st level decomposition: 17071.34551444409 PSNR: 8.472233682832812
MSE for 2nd level decomposition: 17077.19374549213 PSNR: 8.469523524648707
MSE for 3rd level decomposition: 17133.016248341803 PSNR: 8.50033588018306


True